In [1]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
KEY=os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(openai_api_key=KEY,model_name='gpt-3.5-turbo',temperature=0.6)

C:\Users\risha\AppData\Local\Temp\ipykernel_24760\261412575.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=KEY,model_name='gpt-3.5-turbo',temperature=0.6)


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'openai_api_key': None, ...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error